In [1]:
import numpy as np
from scipy.stats import multivariate_normal


class ProbabilityModel:  # Works reliably for 2(+) Dimensional distributions
    """ properties
        modeltype; % multivariate normal ('mvarnorm' - for real coded) or univariate marginal distribution ('umd' - for binary coded)    
        mean_noisy;
        mean_true;
        covarmat_noisy;
        covarmat_true;
        probofone_noisy;
        probofone_true;
        probofzero_noisy;
        probofzero_true;    
        vars;
      end"""

    def __init__(self, modeltype):
        self.modeltype = modeltype
        
    def sample(self, nos):
        # print('nos,self.vars', nos,self.vars)
        nos = int(nos)
        solutions = np.random.multivariate_normal(self.mean_true, self.covarmat_true, size=nos)
        return solutions

    def pdfeval(self, solutions):
        """Calculating the probabilty of every solution
        
        Arguments:
            solutions {[2-D Array]} -- [solution or population of evolutionary algorithm]
        
        Returns:
            [1-D Array] -- [probabilty of every solution]
        """

        mvn = multivariate_normal(self.mean_noisy,self.covarmat_noisy)  
        # create a multivariate Gaussian object with specified mean and covariance matrix
        probofsols = mvn.pdf(solutions)
        return probofsols

    def buildmodel(self, solutions):
        pop, self.vars = solutions.shape
        self.mean_true = np.mean(solutions, 0)
        # Tính ma trận hiệp phương sai của solutions. Ma trận hiệp phương sai có đường chéo chính là phương sai
        # của các mẫu dữ liệu theo từng chiều
        covariance = np.cov(solutions)
        # Simplifying to univariate distribution by ignoring off diagonal terms of covariance matrix
        # Giữ lại đường chéo chính của ma trận hiệp phương sai
        self.covarmat_true = np.diag(np.diag(covariance))
        # Thêm 10% noise để tránh overfit
        self.solutions_noisy = np.append(solutions, np.random.rand(round(0.1 * pop), self.vars), 0)
        self.mean_noisy = np.mean(self.solutions_noisy, 0)
        covariance = np.cov(self.solutions_noisy)
        # Simplifying to univariate distribution by ignoring off diagonal terms of covariance matrix
        self.covarmat_noisy = np.diag(np.diag(covariance))
        self.covarmat_noisy = np.cov(self.solutions_noisy)


In [2]:
import os
from slgep_lib import wrap_config
from utils import Saver
from cea import cea
import argparse
import yaml
from tools import Tools

In [3]:
import gym
import numpy as np


class Taskset:

    def __init__(self, config):
        names = config['names']
        self.config = config
        self.envs = [gym.make(name) for name in names]

    def run_episode(self, sf, policy_function):
        env = self.envs[0]
        observation = env.reset()
        total_reward = 0
        for i in range(18000):
            action = policy_function(observation.astype(np.float32))
            observation, reward, done, info = env.step(action)
            total_reward += reward
            if done:
                break
        return -total_reward

    @property
    def K(self):
        return len(self.config['h_mains'])


In [4]:
import numpy as np
from copy import deepcopy
from collections import namedtuple

ChromosomeRange = namedtuple('ChromosomeRange', ('R1', 'R2', 'R3', 'R4'))
# | --- Function Set --- | --- ADF Set --- | --- ADF Terminal Set --- | --- Terminals --- |
# | 'function_set'       | 'adf_set'       | 'adf_terminal_set'       | 'terminal_set'    |
# |                      |                 |        (Variables)       |     (Inputs)      |
# 0 ---------------------| R1 -------------| R2 ----------------------| R3 ---------------| R4

class Node:

    def __init__(self, index, arity, parent, chromosome_factory):
        self.index = index
        self.arity = arity
        self.parent = parent
        self.children = []
        self.chromosome_factory = chromosome_factory

    def _set_adfs_terminals(self, inputs):
        config = self.chromosome_factory.config
        for i in range(len(config['adf_terminal_set'])):
            config['adf_terminal_set'][i]['value'] = inputs[i]

    def get_value(self):
        config = self.chromosome_factory.config
        # Extract range
        R1, R2, R3, R4 = self.chromosome_factory.chromosome_range
        # If this node is a leaf, return its value
        if self.index >= R3:
            return config['terminal_set'][self.index - R3]['value']
        if self.index >= R2:
            return config['adf_terminal_set'][self.index - R2]['value']
        # If this node is a function or ADF node, 
        # we need to pass in its children as params
        params = []
        for child in self.children:
            value = child.get_value()
            if np.isnan(value):
                return float('nan')
            params.append(child.get_value())
        # If this node is an auto defined function
        # Assign input to the ADF variables
        if self.index >= R1:
            self._set_adfs_terminals(params)
            return config['adf_set'][self.index - R1]['func'].get_value()
        # If this node is a normal function
        function = config['function_set'][self.index]
        return config['function_set'][self.index]['func'](*params)

class ADF:

    def __init__(self, gene, chromosome_factory):
        self.gene = gene
        self.root = None
        self.chromosome_factory = chromosome_factory
        self._parse()

    def _parse(self):
        config = self.chromosome_factory.config
        symbols = config['function_set'] + config['adf_set'] + \
                  config['terminal_set'] + config['adf_terminal_set']

        gene = deepcopy(self.gene).tolist()

        # Assign root
        self.root = Node(index=gene[0],
                         arity=symbols[gene[0]]['arity'],
                         parent=None,
                         chromosome_factory=self.chromosome_factory)
        queue = [self.root]
        gene.pop(0)

        # Traverse BFS to build tree
        while len(queue) and len(gene):
            parent = queue.pop(0)

            for i in range(parent.arity):
                node = Node(index=gene[0],
                            arity=symbols[gene[0]]['arity'],
                            parent=parent,
                            chromosome_factory=self.chromosome_factory)
                queue.append(node)
                gene.pop(0)
                parent.children.append(node)

    def get_value(self):
        return self.root.get_value()

class ChromosomeFactory:

    def __init__(self, _config):
        self.config = _config
        # Assign defined structure of the solution
        config = _config
        # Compute chromosome range
        R1 = len(config['function_set'])
        R2 = R1 + len(config['adf_set'])
        R3 = R2 + len(config['adf_terminal_set'])
        R4 = R3 + len(config['terminal_set'])
        self.chromosome_range = ChromosomeRange(R1, R2, R3, R4)

    def _get_feasible_range(self, i):
        R1, R2, R3, R4 = self.chromosome_range
        config = self.config
        # gene at i belong to one of the given mains
        if i < config['num_main'] * (config['h_main'] + config['l_main']):
            if i % (config['h_main'] + config['l_main']) < config['h_main']:
                # Head of main: adf_set and function_set
                return 0, R2
            else:
                # Tail of main: terminal_set
                return R3, R4
        if (i - config['num_main'] * (config['h_main'] + config['l_main'])) % \
                (config['h_adf'] + config['l_adf']) < config['h_adf']:
            # Head of ADF: function_set
            return 0, R1
        else:
            # Tail of ADF: adf_terminal_set
            return R2, R3

    def initialize(self):
        config = self.config
        population = np.empty([config['pop_size'] * config['K'] * 2, config['dim']])
        for j in range(config['dim']):
            low, high = self._get_feasible_range(j)
            population[:, j] = np.random.randint(low, high, size=config['pop_size'] * config['K'] * 2)
        return population.astype(np.int32)

    def parse(self, chromosome):
        # Parse the auto defined functions
        config = self.config
        for i in range(config['num_adf']):
            head = config['num_main'] * (config['h_main'] + config['l_main']) + \
                   i * (config['h_adf'] + config['l_adf'])
            tail = head + config['h_adf'] + config['l_adf']
            config['adf_set'][i]['func'] = ADF(chromosome[head:tail], self)

        # Parse the main program
        for i in range(config['num_main']):
            head = i * (config['h_main'] + config['l_main'])
            tail = head + config['h_main'] + config['l_main']
            config['main'].append(ADF(chromosome[head:tail], self))

    def _set_main_terminals(self, inputs):
        config = self.config
        for i in range(len(config['terminal_set'])):
            config['terminal_set'][i]['value'] = inputs[i]

    def get_value(self, inputs):
        config = self.config
        self._set_main_terminals(inputs)
        outputs = []
        for i in range(config['num_main']):
            outputs.append(config['main'][i].get_value())
        return outputs

    def get_action(self, inputs):
        config = self.config
        self._set_main_terminals(inputs)
        outputs = []
        for i in range(config['num_main']):
            outputs.append(config['main'][i].get_value())
        outputs = np.array(outputs)
        outputs[np.where(outputs == np.nan)[0]] = -np.inf
        return np.argmax(outputs)

    def one_point_crossover(self, pa, pb):
        D = len(pa)
        index = np.random.randint(low=1, high=D-1)
        ca = np.empty_like(pa)
        cb = np.empty_like(pa)

        ca = np.concatenate([pa[:index], pb[index:]])
        cb = np.concatenate([pb[:index], pb[index:]])
        return ca, cb

    def _get_crossover_range(self, i):
        config = self.config
        n, h, l = config['num_main'], config['h_main'], config['l_main']
        n_adf, h_adf, l_adf = config['num_adf'], config['h_adf'], config['l_adf']
        if i < n * (h + l):
            if i % (l + h) == 0:
                low = (i / (h + l) - 1) * (h + l)
                high = (i / (h + l) + 1) * (h + l)
            else:
                low = np.floor(i / (h + l)) * (h + l)
                high = np.ceil(i / (h + l)) * (h + l)
        else:
            j = i - n * (h + l)
            if j % (l_adf + h_adf) == 0:
                low = (j / (h_adf + l_adf) - 1) * (h_adf + l_adf)
                high = (j / (h_adf + l_adf) + 1) * (h_adf + l_adf)
            else:
                low = np.floor(j / (h_adf + l_adf)) * (h_adf + l_adf)
                high = np.ceil(j / (h_adf + l_adf)) * (h_adf + l_adf)
            low += n * (h + l)
            high += n * (h + l)
        return int(low), int(high)

    def one_point_crossover_adf(self, pa, pb):
        D = len(pa)
        i = np.random.randint(low=1, high=D-1)
        low, high = self._get_crossover_range(i)
        ca = deepcopy(pa)
        cb = deepcopy(pa)
        if np.random.rand() < 0.5:
            ca[low:i] = pb[low:i]
            cb[low:i] = pa[low:i]
        else:
            ca[i:high] = pb[i:high]
            cb[i:high] = pa[i:high]
        return ca, cb

    def one_point_crossover_adf_multitask(self, pa, pb):
        D = len(pa)
        config = self.config
        low = config['num_main'] * (config['h_main'] + config['l_main'])
        i = np.random.randint(low=low + 1, high=D-1)
        low, high = self._get_crossover_range(i)
        ca = deepcopy(pa)
        cb = deepcopy(pa)
        if np.random.rand() < 0.5:
            ca[low:i] = pb[low:i]
            cb[low:i] = pa[low:i]
        else:
            ca[i:high] = pb[i:high]
            cb[i:high] = pa[i:high]
        return ca, cb

    def uniform_mutate(self, p, mutation_rate):
        c = deepcopy(p)
        for i in range(len(p)):
            if np.random.rand() < mutation_rate:
                low, high = self._get_feasible_range(i)
                c[i] = np.random.randint(low, high)
        return c

    def shorten_one_func_of_main(self, p, p_h_main):
        c = deepcopy(p)

        config = self.config
        c_h_main = p_h_main
        c_l_main = c_h_main * (config['max_arity'] - 1) + 1

        max_sum_arity = config['max_arity'] * c_h_main
        R1, R2, R3, R4 = self.chromosome_range

        sub_ind = []

        not_main_part = c[config['num_main']*(c_h_main+c_l_main):config['dim']]

        for i in range(config['num_main']):
            sum_arity = 0
            sub_tree = c[i*(c_h_main+c_l_main):(i+1)*(c_h_main+c_l_main)]

            for j in range(c_h_main):
                if sub_tree[j] < R1: sum_arity += FUNCTION_SET[sub_tree[j]]["arity"]
                else: sum_arity += config["max_arity"]

            last_arity = config["max_arity"]
            if sub_tree[c_h_main - 1] < R1: last_arity = FUNCTION_SET[sub_tree[c_h_main - 1]]["arity"]

            head_del = c_h_main - 1
            tail_del = c_h_main + c_l_main - (max_sum_arity - sum_arity) - last_arity

            sub_tree[head_del] = sub_tree[tail_del]
            sub_tree = np.delete(sub_tree, tail_del)

            sub_h_main = c_h_main - 1
            sub_l_main = sub_h_main * (config['max_arity'] - 1) + 1
            sub_tree = sub_tree[:sub_h_main+sub_l_main]

            sub_ind.extend(sub_tree)

        sub_ind.extend(not_main_part)

        return np.array(sub_ind, dtype=np.int32), sub_h_main


In [5]:
from atari_benchmark import *
from copy import deepcopy
from slgep_lib import *
import numpy as np
import pickle
import yaml
import os


class Evaluator:

    def __init__(self, config):
        self.taskset = Taskset(config)
        self.cf = []
        for h_main in config['h_mains']:
            config['h_main'] = h_main
            config['h_main_multitask'] = np.max(config['h_mains'])
            self.cf.append(ChromosomeFactory(config))

    def decode(self, chromosome, sf):
        config = self.cf[sf].config
        current_h_main = config['h_main_multitask']
        while config['h_main'] < current_h_main:
            chromosome, _ = self.cf[sf].shorten_one_func_of_main(chromosome, current_h_main)
            current_h_main -= 1
        return chromosome

    def evaluate(self, chromosome, sf):
        chromosome = self.decode(chromosome, sf)
        self.cf[sf].parse(chromosome)
        try:
            fitness = self.taskset.run_episode(sf, self.cf[sf].get_action)
        except (OverflowError, ValueError) as e:
            fitness = np.inf
        return fitness


class Saver:

    def __init__(self, config, instance, seed):
        '''Folder result/instance
                            config.yaml
                            <seed>.pkl
        Parameters
        ----------
            config (dict): configuration of the problem
            instance (str): name of the benchmark
        '''
        self.seed = seed
        self.instance = instance
        # Create result folder
        folder = 'result'
        if not os.path.exists(folder):
            os.mkdir(folder)
        folder = 'result/%s' % instance
        if not os.path.exists(folder):
            os.mkdir(folder)
        # Save configuration
        path = os.path.join(folder, 'config.yaml')
        _config = deepcopy(config)
        del _config['function_set']
        del _config['adf_set']
        del _config['adf_terminal_set']
        del _config['terminal_set']
        with open(path, 'w') as fp:
            yaml.dump(_config, fp)
        self.results = []

    def append(self, result):
        self.results.append(result)
        self.save()

    def save(self):
        path = os.path.join('result', self.instance, '%d.pkl' % self.seed)
        with open(path, 'wb') as fp:
            pickle.dump(self.results, fp, protocol=pickle.HIGHEST_PROTOCOL)


In [6]:
import numpy as np


# MULTIFACTORIAL EVOLUTIONARY HELPER FUNCTIONS
def find_relative(population, skill_factor, sf, N):
    return population[np.random.choice(np.where(skill_factor[:N] == sf)[0])]


def calculate_scalar_fitness(factorial_cost):
    return 1 / np.min(np.argsort(np.argsort(factorial_cost, axis=0), axis=0) + 1, axis=1)


# OPTIMIZATION RESULT HELPERS
def get_best_individual(population, factorial_cost, scalar_fitness, skill_factor, sf):
    # select individuals from task sf
    idx = np.where(skill_factor == sf)[0]
    subpop = population[idx]
    sub_factorial_cost = factorial_cost[idx]
    sub_scalar_fitness = scalar_fitness[idx]

    # select best individual
    idx = np.argmax(sub_scalar_fitness)
    x = subpop[idx]
    fun = sub_factorial_cost[idx, sf]
    return x, fun


def get_statistics(factorial_cost, skill_factor, sf):
    idx = np.where(skill_factor == sf)[0]
    sub_factorial_cost = factorial_cost[idx][:, sf]
    return np.mean(sub_factorial_cost), np.std(sub_factorial_cost)


In [7]:
# Load configuration
config = yaml.load(open('config.yaml').read())

# Load benchmark
singletask_benchmark = yaml.load(open('atari_benchmark/singletask-benchmark.yaml').read())
data = singletask_benchmark['single-2']
config.update(data)
config = wrap_config(config)


/home/quanhm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
/home/quanhm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [ ]:
from joblib import Parallel, delayed
from os import cpu_count
from tqdm import trange


taskset = Taskset(config)
# Model
cf = ChromosomeFactory(config)
# Simple parameter
K = taskset.K
config['K'] = K
N = config['pop_size'] * K
T = config['num_iter']
mutation_rate = config['mutation_rate']
# Initialization
population = cf.initialize()
skill_factor = np.array([i % K for i in range(2 * N)])
factorial_cost = np.full([2 * N, K], np.inf)
scalar_fitness = np.empty([2 * N])

# For parallel evaluation
print('[+] Initializing evaluators')
evaluators = [Evaluator(config) for _ in range(2 * N)]

# First evaluation (sequential)
delayed_functions = []
for i in range(2 * N):
    sf = skill_factor[i]
    delayed_functions.append(delayed(evaluators[i].evaluate)(population[i], sf))
fitnesses = Parallel(n_jobs=cpu_count())(delayed_functions)
for i in range(2 * N):
    sf = skill_factor[i]
    factorial_cost[i, sf] = fitnesses[i]
scalar_fitness = calculate_scalar_fitness(factorial_cost)

# Evolve
iterator = trange(T)
for t in range(10):
    # permute current population
    permutation_index = np.random.permutation(N)
    population[:N] = population[:N][permutation_index]
    print('1.',population)
    print('1.',population.shape)
    skill_factor[:N] = skill_factor[:N][permutation_index]
    factorial_cost[:N] = factorial_cost[:N][permutation_index]
    factorial_cost[N:] = np.inf

    # select pair to crossover
    for i in range(0, N, 2):
        # extract parent
        p1 = population[i]
        sf1 = skill_factor[i]
        p2 = find_relative(population, skill_factor, sf1, N)
        # recombine parent
        c1, c2 = cf.one_point_crossover_adf(p1, p2)
        c1 = cf.uniform_mutate(c1, mutation_rate)
        c2 = cf.uniform_mutate(c2, mutation_rate)
        # save child
        population[N + i, :], population[N + i + 1, :] = c1[:], c2[:]
        skill_factor[N + i] = sf1
        skill_factor[N + i + 1] = sf1
    # evaluation
    delayed_functions = []
    for i in range(2 * N):
        sf = skill_factor[i]
        delayed_functions.append(delayed(evaluators[i].evaluate)(population[i], sf))
    fitnesses = Parallel(n_jobs=cpu_count())(delayed_functions)
    for i in range(2 * N):
        sf = skill_factor[i]
        factorial_cost[i, sf] = fitnesses[i]
    scalar_fitness = calculate_scalar_fitness(factorial_cost)

    # sort
    sort_index = np.argsort(scalar_fitness)[::-1]
    population = population[sort_index]
    print('2.',population)
    print('2.',population.shape)
    skill_factor = skill_factor[sort_index]
    factorial_cost = factorial_cost[sort_index]
    scalar_fitness = scalar_fitness[sort_index]

[+] Initializing evaluators


In [12]:
from joblib import Parallel, delayed
from os import cpu_count
from tqdm import trange


taskset = Taskset(config)
# Model
cf = ChromosomeFactory(config)
# Simple parameter
K = taskset.K
config['K'] = K
N = config['pop_size'] * K
T = config['num_iter']
mutation_rate = config['mutation_rate']
# Initialization
population = cf.initialize()
print(cf.chromosome_range)
nos = population.shape[0]
for i in range(nos):  # Leave-one-out cross validation scheme
    x = np.append(population[:i, :], population[i + 1:, :], 0)
    tmodel = ProbabilityModel('mvarnorm')
    tmodel.buildmodel(x)
   

ChromosomeRange(R1=6, R2=14, R3=16, R4=144)


In [14]:
permutation_index = np.random.permutation(10)
print(permutation_index)

[8 4 2 5 0 1 6 3 7 9]


In [15]:
population[:10] = population[:10][permutation_index]
print(population)
print(population.shape)

[[ 3  4  8 ... 14 14 15]
 [ 9  8  9 ... 14 15 14]
 [ 2  6  8 ... 14 14 14]
 ...
 [ 9 12  5 ... 14 14 14]
 [ 6  1  5 ... 14 14 14]
 [11 12  4 ... 14 15 14]]
(20, 108)


In [16]:
covariance = np.cov(population.T)
print(covariance.shape)

(108, 108)


In [17]:
covarmat_true = np.diag(np.diag(covariance))
print(covarmat_true.shape)

(108, 108)


In [18]:
mean = np.mean(population, axis=0)
population.shape, mean.shape, covarmat_true.shape

((20, 108), (108,), (108, 108))

In [64]:
covarmat_true[100,100]

0.0

In [66]:
population2 = cf.initialize()
mvn = multivariate_normal.pdf(population2, mean, covarmat_true, allow_singular = True)
mvn, mvn.shape


(array([1.81369506e-125, 4.41986562e-123, 5.30403398e-120, 1.80457011e-124,
        2.70082114e-119, 2.20562926e-122, 3.13500363e-122, 9.84538409e-121,
        1.98389102e-123, 9.59908932e-125, 3.17202993e-120, 3.41729668e-124,
        1.40453321e-124, 1.23472228e-123, 1.09803900e-127, 5.98384925e-124,
        6.64505061e-125, 2.90967299e-124, 3.25178003e-123, 2.33148294e-124]),
 (20,))

In [67]:
np.diag(covariance)

array([1.58000000e+01, 1.36289474e+01, 1.17263158e+01, 1.76947368e+01,
       1.15684211e+01, 1.98815789e+01, 1.74708158e+03, 1.76920000e+03,
       1.62536842e+03, 8.88765789e+02, 1.00595789e+03, 1.26087105e+03,
       1.43106316e+03, 1.38815789e+01, 1.36289474e+01, 2.04500000e+01,
       1.51026316e+01, 1.71052632e+01, 1.84736842e+01, 1.78827368e+03,
       8.66576316e+02, 1.16946316e+03, 1.11983947e+03, 1.42573421e+03,
       1.46478947e+03, 2.02866053e+03, 1.94631579e+01, 1.18184211e+01,
       1.29763158e+01, 1.24710526e+01, 1.33263158e+01, 2.02394737e+01,
       9.80513158e+02, 1.30101053e+03, 6.32576316e+02, 7.76765789e+02,
       1.33466053e+03, 1.35674737e+03, 1.17546316e+03, 2.42105263e+01,
       1.09368421e+01, 1.52526316e+01, 1.71578947e+01, 1.43263158e+01,
       1.41973684e+01, 1.56693684e+03, 9.92431579e+02, 9.95944737e+02,
       1.45651579e+03, 1.01925000e+03, 1.26630263e+03, 1.07204211e+03,
       3.31315789e+00, 1.62894737e+00, 2.76578947e+00, 2.39473684e-01,
      

In [68]:
np.random.normal(0, 0.01, 4)

array([-0.01058315,  0.02262643, -0.01066715,  0.00271562])

In [69]:
np.random.permutation(10)

array([5, 0, 2, 1, 9, 8, 4, 3, 7, 6])

In [71]:
np.round([0.2,0.7])

array([0., 1.])

In [74]:
np.random.randint(0,2,10)

array([0, 0, 1, 1, 0, 1, 0, 0, 1, 0])

In [79]:
np.random.multivariate_normal([2,3], [[3,3],[3,3]])[0]

0.6193456206299983

In [84]:
np.array([1.742e-1]).astype(float64)

NameError: name 'float64' is not defined